In [7]:
df = spark.sql("SELECT * FROM streamingpulse.web_cookies")

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 9, Finished, Available, Finished)

In [8]:
from pyspark.sql.functions import *
df2=df.withColumn("page_url",regexp_replace(col("page_url"), r"[^a-zA-Z0-9- ]", "")) \
       .withColumn("timestamp",translate(col('timestamp'),'TZ',' ').cast('timestamp')) \
       .withColumnRenamed("utm_source","Platform") 
display(df2)

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 36968bd2-57fb-4702-82d4-2b286efc8fe5)

In [9]:
df3=df2.withColumn("events",explode(col('events')))

df4=df3.withColumn('event_type',col("events.event_type")).drop('events')

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 11, Finished, Available, Finished)

In [10]:
df5=df4.select("session_id","user_id","timestamp","device_type","os",
"browser","page_url","referrer","Platform","event_type","time_spent_seconds","is_returning_visitor","visit_count")

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 12, Finished, Available, Finished)

In [11]:
df6=df5.groupBy("user_id").agg(sum('visit_count').alias("visitor_count"))
display(df6)

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9b038f7b-659f-46b5-a26c-38cf87ed8805)

In [12]:
df6.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("visit_count")

df5.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("final_cookies")

StatementMeta(, afb6ccaf-c6c0-4606-a1ee-eea006dadc49, 14, Finished, Available, Finished)